In [2]:
import dash
from dash import Dash, dcc, html, Input, Output
import pandas as pd
import os
import requests
import numpy as np
from itertools import product

os.getcwd()


import os
import sys

# Get the current working directory
notebook_path = os.getcwd()
# Add the 'functions' directory to the Python path
sys.path.append('/home/jovyan/Bordtennis/src/functions')

In [32]:
players = {'august': {'rating': 966.8799018458224, 'games_played': 48},
 'eigil': {'rating': 977.763720703288, 'games_played': 67},
 'joel': {'rating': 1205.3617582190896, 'games_played': 65},
 'jonas': {'rating': 998.643415503727, 'games_played': 51},
 'jørgen': {'rating': 638.1919145202676, 'games_played': 63},
 'kjetil': {'rating': 1213.159289207805, 'games_played': 56}}

match_history =[]

remaining_matches =  [
                     ('jørgen', 'kjetil'),
                     ('kjetil', 'august'),
                     ('jonas', 'jørgen'),
                     ('eigil', 'jonas'),
                     ('august', 'jørgen'),
                     ('kjetil', 'jonas'),
                     ('eigil', 'jørgen'),
                     ('august', 'jonas'),
                     ('eigil', 'kjetil'),
                     ('eigil', 'august'),
                     ('joel', 'eigil'),
                     ('joel', 'kjetil'),
                     ('joel', 'august'),
                     ('joel', 'jonas'),
                     ('joel', 'jørgen')]




In [45]:
from itertools import product

def calculate_rating(rating_winner, rating_loser):
    k_factor = 32  # You can adjust this value based on your needs

    expected_winner = 1 / (1 + 10 ** ((rating_loser - rating_winner) / 400))
    expected_loser = 1 / (1 + 10 ** ((rating_winner - rating_loser) / 400))

    new_rating_winner = rating_winner + k_factor * (1 - expected_winner)
    new_rating_loser = rating_loser + k_factor * (0 - expected_loser)

    return new_rating_winner, new_rating_loser


def update_ratings(players, match_history, winner, loser):
    winner_rating = players[winner]["rating"]
    loser_rating = players[loser]["rating"]

    new_winner_rating, new_loser_rating = calculate_rating(winner_rating, loser_rating)

    players[winner]["rating"] = new_winner_rating
    players[loser]["rating"] = new_loser_rating

    players[winner]["games_played"] += 1
    players[loser]["games_played"] += 1

    match_history.append({"winner": winner, "loser": loser})


def simulate_matches(players, remaining_matches):
    scenarios = []

    def generate_scenarios(matches, current_scenario):
        if not matches:
            scenarios.append(current_scenario.copy())
            return

        match = matches[0]
        for outcome in [(match[0], match[1]), (match[1], match[0])]:
            update_ratings(players, [], outcome[0], outcome[1])
            current_scenario.append(outcome)
            generate_scenarios(matches[1:], current_scenario)
            current_scenario.pop()
            # Reset players' ratings and games played after each iteration
            players[outcome[0]]["rating"] = initial_ratings[outcome[0]]
            players[outcome[1]]["rating"] = initial_ratings[outcome[1]]
            players[outcome[0]]["games_played"] -= 1
            players[outcome[1]]["games_played"] -= 1

    initial_ratings = {player: players[player]["rating"] for player in players}
    generate_scenarios(remaining_matches, [])
    return scenarios

def highest_rating(players):
    # Function to get the player with the highest rating
    highest_rating_player = max(players, key=lambda x: players[x]['rating'])
    return highest_rating_player


scenarios = simulate_matches(players, remaining_matches)

count_stein_not_highest = 0

def print_ratings(players):
    for player, stats in players.items():
        print(f"{player}: Rating - {stats['rating']}, Games Played - {stats['games_played']}")

scenarios = simulate_matches(players, remaining_matches)

for i, scenario in enumerate(scenarios, 1):
    print(f"Scenario {i}:")
    for match in scenario:
        print(f"{match[0]} vs {match[1]}")
    
    print("\nPlayer Ratings before scenario:")
    print_ratings(players)
    
    initial_ratings = {player: players[player]["rating"] for player in players}
    initial_games_played = {player: players[player]["games_played"] for player in players}

    for match in scenario:
        update_ratings(players, [], match[0], match[1])

    print("\nPlayer Ratings after scenario:")
    print_ratings(players)
    
    # Revert ratings and games played to initial values
    for player in players:
        players[player]["rating"] = initial_ratings[player]
        players[player]["games_played"] = initial_games_played[player]
    
    print("\n")

Scenario 1:
august vs eigil
jørgen vs eigil
stein vs eigil
jonas vs eigil
jørgen vs august
jørgen vs stein
jørgen vs jonas
august vs jonas
august vs stein
jonas vs stein

Player Ratings before scenario:
august: Rating - 954.7204990825949, Games Played - 24
eigil: Rating - 1080.9219453239289, Games Played - 38
zeek: Rating - 1000, Games Played - 0
jonas: Rating - 984.7795600622975, Games Played - 19
jørgen: Rating - 819.2114123992312, Games Played - 37
stein: Rating - 1160.0822459098308, Games Played - 26

Player Ratings after scenario:
august: Rating - 995.0699451410491, Games Played - 28
eigil: Rating - 1005.5776305175224, Games Played - 42
zeek: Rating - 1000, Games Played - 0
jonas: Rating - 987.1672408523988, Games Played - 23
jørgen: Rating - 914.6835683543299, Games Played - 41
stein: Rating - 1097.217277912583, Games Played - 30


Scenario 2:
august vs eigil
jørgen vs eigil
stein vs eigil
jonas vs eigil
jørgen vs august
jørgen vs stein
jørgen vs jonas
august vs jonas
august vs s

In [33]:
from ratings_functions import calculate_rating, update_ratings, simulate_matches, highest_rating



scenarios = simulate_matches(players, remaining_matches)

scenarios_data = []
for i, scenario in enumerate(scenarios, 1):
    scenario_info = {"Scenario": f"Scenario {i}"}
    matches_info = []
    
    # Store match information
    for match in scenario:
        matches_info.append(f"{match[0]} vs {match[1]}")
    scenario_info["Matches"] = matches_info
    
    # Store player ratings before scenario
    player_ratings_before = []
    for player, stats in players.items():
        player_ratings_before.append(f"{player}: Rating - {stats['rating']}, Games Played - {stats['games_played']}")
    scenario_info["Player Ratings Before"] = player_ratings_before
    
    # Store player ratings after scenario
    initial_ratings = {player: players[player]["rating"] for player in players}
    initial_games_played = {player: players[player]["games_played"] for player in players}

    for match in scenario:
        update_ratings(players, [], match[0], match[1])

    player_ratings_after = []
    for player, stats in players.items():
        player_ratings_after.append(f"{player}: Rating - {stats['rating']}, Games Played - {stats['games_played']}")
    scenario_info["Player Ratings After"] = player_ratings_after
    
    # Revert ratings and games played to initial values
    for player in players:
        players[player]["rating"] = initial_ratings[player]
        players[player]["games_played"] = initial_games_played[player]
    
    scenarios_data.append(scenario_info)

# Create DataFrame
scenarios_df = pd.DataFrame(scenarios_data)


In [34]:
print(scenarios_df)

             Scenario                                            Matches  \
0          Scenario 1  [jørgen vs kjetil, kjetil vs august, jonas vs ...   
1          Scenario 2  [jørgen vs kjetil, kjetil vs august, jonas vs ...   
2          Scenario 3  [jørgen vs kjetil, kjetil vs august, jonas vs ...   
3          Scenario 4  [jørgen vs kjetil, kjetil vs august, jonas vs ...   
4          Scenario 5  [jørgen vs kjetil, kjetil vs august, jonas vs ...   
...               ...                                                ...   
32763  Scenario 32764  [kjetil vs jørgen, august vs kjetil, jørgen vs...   
32764  Scenario 32765  [kjetil vs jørgen, august vs kjetil, jørgen vs...   
32765  Scenario 32766  [kjetil vs jørgen, august vs kjetil, jørgen vs...   
32766  Scenario 32767  [kjetil vs jørgen, august vs kjetil, jørgen vs...   
32767  Scenario 32768  [kjetil vs jørgen, august vs kjetil, jørgen vs...   

                                   Player Ratings Before  \
0      [august: Rating - 96

In [35]:
ratings_before_df = scenarios_df['Player Ratings Before'].apply(lambda x: [float(s.split(':')[1].split(',')[0].strip().split()[2]) for s in x]).apply(pd.Series)

player_names = [s.split(':')[0] for s in scenarios_df['Player Ratings Before'].iloc[0]]

# Rename the columns of the DataFrame
ratings_before_df.columns = [f"{name} Before" for name in player_names]

scenarios_df = pd.concat([scenarios_df, ratings_before_df], axis=1)

In [36]:
ratings_after_df = scenarios_df['Player Ratings After'].apply(lambda x: [float(s.split(':')[1].split(',')[0].strip().split()[2]) for s in x]).apply(pd.Series)

player_names = [s.split(':')[0] for s in scenarios_df['Player Ratings After'].iloc[0]]

# Rename the columns of the DataFrame
ratings_after_df.columns = [f"{name} after" for name in player_names]

scenarios_df = pd.concat([scenarios_df, ratings_after_df], axis=1)

In [38]:
# Filter rows where any of the "after" columns except 'stein after' are higher than 'stein Before'
x= 'kjetil after'

filtered_df = scenarios_df[
    (scenarios_df['kjetil after'] > scenarios_df[x]) |
    (scenarios_df['jørgen after'] > scenarios_df[x]) |
    (scenarios_df['jonas after'] > scenarios_df[x]) |
    (scenarios_df['eigil after'] > scenarios_df[x]) |
    (scenarios_df['august after'] > scenarios_df[x]) |
    (scenarios_df['joel after'] > scenarios_df[x]) 
]

filtered_df

,Scenario,Matches,Player Ratings Before,Player Ratings After,august Before,eigil Before,joel Before,jonas Before,jørgen Before,kjetil Before,august after,eigil after,joel after,jonas after,jørgen after,kjetil after
0,Scenario 1,"[jørgen vs kjetil, kjetil vs august, jonas vs ...","[august: Rating - 966.8799018458224, Games Pla...","[august: Rating - 961.3711053553695, Games Pla...",966.879902,977.763721,1205.361758,998.643416,638.191915,1213.159289,961.371105,1028.488589,1240.688926,955.971068,654.799527,1158.680785
1,Scenario 2,"[jørgen vs kjetil, kjetil vs august, jonas vs ...","[august: Rating - 966.8799018458224, Games Pla...","[august: Rating - 961.3711053553695, Games Pla...",966.879902,977.763721,1205.361758,998.643416,638.191915,1213.159289,961.371105,1028.488589,1208.688926,955.971068,686.799527,1158.680785
2,Scenario 3,"[jørgen vs kjetil, kjetil vs august, jonas vs ...","[august: Rating - 966.8799018458224, Games Pla...","[august: Rating - 961.3711053553695, Games Pla...",966.879902,977.763721,1205.361758,998.643416,638.191915,1213.159289,961.371105,1028.488589,1208.897436,987.971068,654.591017,1158.680785
3,Scenario 4,"[jørgen vs kjetil, kjetil vs august, jonas vs ...","[august: Rating - 966.8799018458224, Games Pla...","[august: Rating - 961.3711053553695, Games Pla...",966.879902,977.763721,1205.361758,998.643416,638.191915,1213.159289,961.371105,1028.488589,1176.897436,987.971068,686.591017,1158.680785
4,Scenario 5,"[jørgen vs kjetil, kjetil vs august, jonas vs ...","[august: Rating - 966.8799018458224, Games Pla...","[august: Rating - 993.3711053553695, Games Pla...",966.879902,977.763721,1205.361758,998.643416,638.191915,1213.159289,993.371105,1028.488589,1209.783162,955.079036,654.597323,1158.680785
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32744,Scenario 32745,"[kjetil vs jørgen, august vs kjetil, jørgen vs...","[august: Rating - 966.8799018458224, Games Pla...","[august: Rating - 959.1298877002704, Games Pla...",966.879902,977.763721,1205.361758,998.643416,638.191915,1213.159289,959.129888,908.316275,1209.791794,1016.237061,717.841984,1188.682999
32752,Scenario 32753,"[kjetil vs jørgen, august vs kjetil, jørgen vs...","[august: Rating - 966.8799018458224, Games Pla...","[august: Rating - 959.1751068500673, Games Pla...",966.879902,977.763721,1205.361758,998.643416,638.191915,1213.159289,959.175107,940.316275,1211.147789,1016.288297,717.855436,1155.217096
32753,Scenario 32754,"[kjetil vs jørgen, august vs kjetil, jørgen vs...","[august: Rating - 966.8799018458224, Games Pla...","[august: Rating - 959.1751068500673, Games Pla...",966.879902,977.763721,1205.361758,998.643416,638.191915,1213.159289,959.175107,940.316275,1179.147789,1016.288297,749.855436,1155.217096
32754,Scenario 32755,"[kjetil vs jørgen, august vs kjetil, jørgen vs...","[august: Rating - 966.8799018458224, Games Pla...","[august: Rating - 959.1751068500673, Games Pla...",966.879902,977.763721,1205.361758,998.643416,638.191915,1213.159289,959.175107,940.316275,1179.487873,1048.288297,717.515352,1155.217096


In [23]:
players_to_check = ['kjetil after', 'joel after']

# Initialize a boolean series to store the filter condition
condition = pd.Series([False] * len(scenarios_df))

# Apply the filtering for each column in columns_to_check
for col in players_to_check:
    condition |= (
        #(scenarios_df['kjetil after'] > scenarios_df[col]) |
        (scenarios_df['jørgen after'] > scenarios_df[col]) |
        (scenarios_df['jonas after'] > scenarios_df[col]) |
        (scenarios_df['eigil after'] > scenarios_df[col]) |
        (scenarios_df['august after'] > scenarios_df[col])
        #(scenarios_df['joel after'] > scenarios_df[col])
    )
    
    


scenarios_df[condition]
scenarios_df.loc[condition, ['eigil after', 'jørgen after', 'jonas after', 'kjetil after', 'joel after', 'august after]] 

,eigil after,jørgen after,jonas after,kjetil after,joel after,august after
33439,1058.475726,686.088911,1070.105902,1159.357474,1062.520328,963.451659
33471,1027.696758,686.089140,1070.106737,1159.301271,1062.537924,994.268171
33503,1029.079481,686.092540,1070.119153,1189.832021,1062.799732,962.077073
33535,998.207866,686.093736,1070.123523,1189.780332,1062.891896,992.902647
33695,1030.122760,717.628540,1070.055001,1158.222993,1061.894269,962.076438
...,...,...,...,...,...,...
261919,1052.557201,749.192388,1015.246283,1156.042583,1036.250411,990.711134
262047,1051.174712,749.197176,1045.782165,1156.039828,1036.488848,961.317271
262079,1020.463876,749.197961,1045.784041,1155.980659,1036.527990,992.045473
262111,1021.850125,749.201249,1045.791896,1186.526921,1036.691902,959.937906


In [12]:
filtered_df['Matches'].loc[8]

['jonas vs jørgen',
 'stein vs august',
 'stein vs jørgen',
 'eigil vs jonas',
 'august vs jørgen',
 'stein vs jonas',
 'eigil vs jørgen',
 'august vs jonas',
 'eigil vs stein',
 'eigil vs august',
 'zeek vs eigil',
 'stein vs zeek',
 'zeek vs august',
 'zeek vs jonas',
 'zeek vs jørgen']

In [31]:
matches_list = [
    'jonas vs jørgen',
    'stein vs august',
    'stein vs jørgen',
    'eigil vs jonas',
    'august vs jørgen',
    'stein vs jonas',
    'eigil vs jørgen',
    'august vs jonas',
    'eigil vs stein',
    'eigil vs august'
]

# Filter the DataFrame where 'Matches' column values are in the matches_list
idag = scenarios_df[scenarios_df['Matches'].isin(matches_list)]
idag

,Scenario,Matches,Player Ratings Before,Player Ratings After,august Before,eigil Before,zeek Before,jonas Before,jørgen Before,stein Before,august after,eigil after,zeek after,jonas after,jørgen after,stein after
